In [1]:

from kmeans_pytorch import kmeans
import h5py
import matplotlib.pyplot as plt
import numpy as np
import argparse
import importlib
import random
import os
import FLAlgorithms
import utils

#for jupyter notebook reload modules
import imp
imp.reload(FLAlgorithms) 
imp.reload(utils) 
from FLAlgorithms.servers.serveravg import FedAvg
from FLAlgorithms.servers.serverpFedMe import pFedMe
from FLAlgorithms.servers.serverperavg import PerAvg
from FLAlgorithms.trainmodel.models import *
from FLAlgorithms.servers.servertrans import pFedTrans, Cluster

from utils.plot_utils import *
from kmeans_pytorch import kmeans

import torch
import copy
torch.manual_seed(0)


In [2]:
def props(cls):   
  return [i for i in cls.__dict__.keys() if i[:1] != '_']

%load_ext autoreload
%autoreload 2

In [3]:

gpu = 0
device = torch.device("cuda:{}".format(gpu) if torch.cuda.is_available() and gpu != -1 else "cpu")
model = DNN(60, 20, 10).to(device)

In [4]:
for value in model.state_dict().values():
    print(value.size())

torch.Size([20, 60])
torch.Size([20])
torch.Size([10, 20])
torch.Size([10])


In [6]:
import time
gpu = 0
device = torch.device("cuda:{}".format(gpu) if torch.cuda.is_available() and gpu != -1 else "cpu")

print("---------------Running time:------------",)
# Generate model
model = 'mclr'
dataset = 'Mnist'

model = Mclr_Logistic().to(device), model
algorithm = "pFedTrans"
# select algorithm
batch_size = 20
learning_rate = 0.01
personal_learning_rate = 0.01
beta = 2
lamda = 15
num_glob_iters = 800
local_epochs = 20
optimizer = "SGD"
numusers = 100 
K = 5
time1 = time.time()
server = pFedTrans(device, dataset, algorithm, model, batch_size, learning_rate,beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, K, personal_learning_rate, 5)
time2 = time.time()
cost_time = time2-time1
        

---------------Running time:------------
Number of users / total users: 100  /  20
Finished creating pFedMe server.


In [7]:
glob_iter = 0

In [8]:
server.evaluate()
server.prev_per_values = []
            
server.attn_optimizer.zero_grad()

print(server.users[0].model.state_dict().values())
for i, user in enumerate(server.users):
    if glob_iter != 0:
        user.prev_per_values = [0]*len(user.per_values)
        server.copy_value(user.per_values, user.prev_per_values, if_grad=True)
    user.train(server.local_epochs)
    #get user embedding vec
    user.emb(server.emb_layer)

print(server.users[0].model.state_dict().values())
server.evaluate()
 

Average Global Accurancy:  0.08342332613390929
Average Global Trainning Accurancy:  0.08360418923799205
Average Global Trainning Loss:  2.418263764276363
odict_values([tensor([[-0.0202, -0.0072,  0.0322,  ...,  0.0074,  0.0303, -0.0296],
        [-0.0186,  0.0064, -0.0356,  ...,  0.0092, -0.0141,  0.0014],
        [ 0.0140, -0.0160, -0.0340,  ..., -0.0065,  0.0020,  0.0101],
        ...,
        [ 0.0123, -0.0354,  0.0271,  ..., -0.0067,  0.0086, -0.0130],
        [ 0.0090,  0.0256,  0.0047,  ..., -0.0015,  0.0345,  0.0252],
        [ 0.0072, -0.0231, -0.0263,  ...,  0.0135, -0.0356,  0.0076]]), tensor([-0.0043,  0.0244, -0.0186,  0.0201, -0.0008,  0.0112, -0.0338, -0.0048,
        -0.0224,  0.0197])])
odict_values([tensor([[-0.0202, -0.0072,  0.0322,  ...,  0.0074,  0.0303, -0.0296],
        [-0.0186,  0.0064, -0.0356,  ...,  0.0092, -0.0141,  0.0014],
        [ 0.0140, -0.0160, -0.0340,  ..., -0.0065,  0.0020,  0.0100],
        ...,
        [ 0.0123, -0.0354,  0.0271,  ..., -0.0067, 

In [9]:
server.form_cluster()
for cluster in server.clusters:
    print("===================================")
    print("cluster users:")
    for user in cluster.users:
        print(user.id)

running k-means on cpu..


[running kmeans]: 3it [00:00, 503.03it/s, center_shift=0.000000, iteration=3, tol=0.000100]

cluster users:
f_00006
f_00007
f_00016
cluster users:
f_00005
f_00015
cluster users:
f_00008
f_00009
f_00018
f_00019
cluster users:
f_00003
f_00013
cluster users:
f_00011
cluster users:
f_00001
cluster users:
f_00002
f_00012
cluster users:
f_00017
cluster users:
f_00004
f_00014
cluster users:
f_00000
f_00010


In [65]:
print(server.users[0].model.state_dict())

OrderedDict([('fc1.weight', tensor([[-0.0003,  0.0191, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0237,  0.0103, -0.0219,  ...,  0.0088, -0.0009,  0.0009],
        [ 0.0144, -0.0336, -0.0346,  ..., -0.0222, -0.0025, -0.0138],
        [-0.0195, -0.0118,  0.0230,  ..., -0.0202,  0.0172,  0.0355]],
       device='cuda:0')), ('fc1.bias', tensor([ 0.0421,  0.0745, -0.0228, -0.0464, -0.0416, -0.0250,  0.0211,  0.0023,
         0.0199,  0.0073], device='cuda:0'))])


In [12]:
model_path = "ne"
i = 10

alg = "pfe"
local_ep1 = 10
print("s{}".format(alg,local_ep1))

spfe


In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import argparse
import importlib
import random
import os
from utils.plot_utils import *
import torch
torch.manual_seed(0)
flag = 1
if(flag==0): # plot for MNIST convex 
    numusers = 5
    num_glob_iters = 800
    dataset = "Mnist"
    local_ep = [20,20,20,20]
    lamda = [15,15,15,15]
    learning_rate = [0.005, 0.005, 0.005, 0.005]
    beta =  [1.0, 1.0, 0.001, 1.0]
    batch_size = [20,20,20,20]
    K = [5,5,5,5,5,5]
    personal_learning_rate = [0.1,0.1,0.1,0.1]
    algorithms = [ "pFedMe_p","pFedMe","PerAvg_p","FedAvg"]
    plot_summary_one_figure_mnist_Compare(num_users=numusers, loc_ep1=local_ep, Numb_Glob_Iters=num_glob_iters, lamb=lamda,
                               learning_rate=learning_rate, beta = beta, algorithms_list=algorithms, batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate)

if(flag==1): # plot for Synthetic covex
    numusers = 10
    num_glob_iters = 600
    dataset = "Synthetic"
    local_ep = [20,20,20]
    lamda = [30,30,30]
    learning_rate = [0.005, 0.005, 0.005]
    beta =  [2.0, 2.0, 2.0]
    batch_size = [20,20,20]
    K = [5,5,5]
    personal_learning_rate = [0.01,0.01,0.01] 
    algorithms = [ "pFedMe_p","pFedMe","pFedTrans_p"]
    plot_summary_one_figure_synthetic_Compare(num_users=numusers, loc_ep1=local_ep, Numb_Glob_Iters=num_glob_iters, lamb=lamda,
                               learning_rate=learning_rate, beta = beta, algorithms_list=algorithms, batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate)



In [67]:

print(server.users[0].model.state_dict().values())
for cluster in server.clusters:
    for user in cluster.users:
        alpha = server.alpha_layer(user.emb_vec)
        alpha = torch.sigmoid(alpha)
        user.per_values = server.weighted_agg_model([cluster.per_values, user.per_values], [1-alpha,alpha])
        user.merge_base_per_model()
    cluster.merge_base_per_model()

print(server.users[0].model.state_dict().values())
server.evaluate_personalized_model()


odict_values([tensor([[-0.0003,  0.0191, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0237,  0.0103, -0.0219,  ...,  0.0088, -0.0009,  0.0009],
        [ 0.0144, -0.0336, -0.0346,  ..., -0.0222, -0.0025, -0.0138],
        [-0.0195, -0.0118,  0.0230,  ..., -0.0202,  0.0172,  0.0355]],
       device='cuda:0'), tensor([ 0.0421,  0.0745, -0.0228, -0.0464, -0.0416, -0.0250,  0.0211,  0.0023,
         0.0199,  0.0073], device='cuda:0')])
odict_values([tensor([[-0.0003,  0.0191, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0237,  0.0103, -0.0219,  ...,  0.0088, -0.0009,  0.0009],
        [ 0.0144, -0.0336, -0.0346,  ..., -0.0222, -0.0025, -0.0138],
        [-0.0195, -0.011

In [32]:
for i, user in enumerate(server.users):
    user.prev_per_values = [0]*len(user.per_values)
    server.copy_value(user.per_values, user.prev_per_values, if_grad=True)

In [33]:
print(server.users[0].prev_per_values)

[tensor([[ 0.0323,  0.0191,  0.0091,  ..., -0.0150,  0.0041,  0.0090],
        [ 0.0253, -0.0168,  0.0191,  ...,  0.0348, -0.0115, -0.0151],
        [-0.0091,  0.0183,  0.0120,  ...,  0.0353, -0.0333,  0.0139],
        ...,
        [ 0.0158, -0.0343, -0.0147,  ..., -0.0248,  0.0140,  0.0297],
        [-0.0180,  0.0150,  0.0291,  ...,  0.0168, -0.0131,  0.0281],
        [-0.0081, -0.0231, -0.0033,  ...,  0.0317, -0.0223, -0.0356]],
       device='cuda:0'), tensor([-0.0037,  0.0056,  0.0282, -0.0248,  0.0320,  0.0182,  0.0168, -0.0324,
         0.0074,  0.0250], device='cuda:0')]


In [3]:
import time
time1 = time.time()
time2 = tijkjme.time()
cost_time = time2-time1 
print("ep cost time : {}", str(cost_time))

ep cost time : {} 0.0


In [34]:
for i, user in enumerate(server.users):
    #if g_iter != 0:
        #user.prev_per_values = [0]*len(user.per_values)
        #server.copy_value(user.prev_per_values, user.per_values)
    user.train(server.local_epochs)
    #get user embedding vec
    user.emb(server.emb_layer)

In [35]:
user = server.users[0]
print(user.prev_per_values)
print(user.per_values)

[tensor([[ 0.0323,  0.0191,  0.0091,  ..., -0.0150,  0.0041,  0.0090],
        [ 0.0253, -0.0168,  0.0191,  ...,  0.0348, -0.0115, -0.0151],
        [-0.0091,  0.0183,  0.0120,  ...,  0.0353, -0.0333,  0.0139],
        ...,
        [ 0.0158, -0.0343, -0.0147,  ..., -0.0248,  0.0140,  0.0297],
        [-0.0180,  0.0150,  0.0291,  ...,  0.0168, -0.0131,  0.0281],
        [-0.0081, -0.0231, -0.0033,  ...,  0.0317, -0.0223, -0.0356]],
       device='cuda:0'), tensor([-0.0037,  0.0056,  0.0282, -0.0248,  0.0320,  0.0182,  0.0168, -0.0324,
         0.0074,  0.0250], device='cuda:0')]
[tensor([[ 0.0323,  0.0191,  0.0091,  ..., -0.0150,  0.0041,  0.0090],
        [ 0.0253, -0.0168,  0.0191,  ...,  0.0348, -0.0115, -0.0151],
        [-0.0091,  0.0183,  0.0120,  ...,  0.0353, -0.0333,  0.0139],
        ...,
        [ 0.0158, -0.0343, -0.0147,  ..., -0.0248,  0.0140,  0.0297],
        [-0.0180,  0.0150,  0.0291,  ...,  0.0168, -0.0131,  0.0281],
        [-0.0081, -0.0231, -0.0033,  ...,  0.0316, 

In [36]:
criteria = nn.MSELoss()
total_train = 0
server.attn_optimizer.zero_grad()
for user in server.users:
    total_train += user.train_samples
targets = None 
preds = None
user = server.users[0]
if preds == None and targets == None:
    preds =  server.copy_value(nn.utils.parameters_to_vector(user.prev_per_values).unsqueeze(0), if_grad=True, if_tensor=True)
    targets =  server.copy_value(nn.utils.parameters_to_vector(server.copy_value(user.per_values)).unsqueeze(0), if_tensor=True)
else:
    preds =  torch.cat((preds, nn.utils.parameters_to_vector(user.prev_per_values).unsqueeze(0)), 0)
    targets =  torch.cat((targets, nn.utils.parameters_to_vector(server.copy_value(user.per_values)).unsqueeze(0)), 0)

#for i, user in enumerate(server.users):

    #ratio = user.train_samples / total_train
    #if preds == None and targets == None:
        #preds =  nn.utils.parameters_to_vector(user.prev_per_values).unsqueeze(0) 
        #targets =  nn.utils.parameters_to_vector(server.copy_value(user.per_values)).unsqueeze(0) 
    #else:
        #preds =  torch.cat((preds, nn.utils.parameters_to_vector(user.prev_per_values).unsqueeze(0)), 0)
        #targets =  torch.cat((targets, ratio * nn.utils.parameters_to_vector(server.copy_value(user.per_values)).unsqueeze(0)), 0)
    #if i >= 5:
        #break

print(preds.size(), targets)
loss = criteria(preds, targets)
loss.backward()
server.attn_optimizer.step()


torch.Size([1, 7850]) tensor([[ 0.0323,  0.0191,  0.0091,  ..., -0.0392,  0.0011,  0.0181]],
       device='cuda:0')


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
server.attn_optimize()

In [37]:
if g_iter % 5 == 0:
    server.form_cluster()

running k-means on cuda:0..


[running kmeans]: 2it [00:00, 511.16it/s, center_shift=0.000000, iteration=2, tol=0.000100]


In [38]:
for cluster in server.clusters:
    cluster.avg_update_model()
    cluster.emb(server.emb_layer)

In [39]:
for cluster in server.clusters:
    server.intra_cluster_agg(cluster)

In [40]:
server.inter_cluster_agg()

In [24]:
user = server.users[1]
print(user.per_values)

[tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0237,  0.0103, -0.0219,  ...,  0.0088, -0.0009,  0.0009],
        [ 0.0144, -0.0336, -0.0346,  ..., -0.0222, -0.0025, -0.0138],
        [-0.0195, -0.0118,  0.0230,  ..., -0.0202,  0.0172,  0.0355]],
       device='cuda:0', grad_fn=<AddBackward0>), tensor([-0.0200,  0.0342,  0.0464, -0.0196, -0.0387, -0.0235,  0.0221,  0.0032,
         0.0200,  0.0073], device='cuda:0', grad_fn=<AddBackward0>)]


In [41]:
for cluster in server.clusters:
    for user in cluster.users:
        alpha = server.alpha_layer(user.emb_vec)
        alpha = torch.sigmoid(alpha)
        user.per_values = server.weighted_agg_model([cluster.per_values, user.per_values], [1-alpha,alpha], use_grad=True)
        user.merge_base_per_model()
    cluster.merge_base_per_model()


In [42]:
server.evaluate()

Average Global Accurancy:  0.828023758099352
Average Global Trainning Accurancy:  0.8423618634886241
Average Global Trainning Loss:  1.2799933238251626


In [43]:
g_iter += 1

In [ ]:
import graphviz
import torchviz
from torchviz import make_dot
import os
os.environ["PATH"] += os.pathsep + 'E:\工具软件\graphviz\\bin'
print(os.environ["PATH"])
dot = make_dot(loss)

In [ ]:
loss.backward()

In [ ]:
loss.backward()